## 2022 CVE Data Review Intro

In [1]:
%%capture
!mkdir -p jsondata
%cd jsondata
!rm *.json 
!rm *.zip 
!wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{2002..2022}.json.zip 
!unzip -o "*.zip" 

In [2]:

from IPython.core.magic import register_cell_magic
from IPython.display import Markdown
import calplot
import datetime
from datetime import date
import glob
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import calendar

@register_cell_magic
def markdown(line, cell):
    return Markdown(cell.format(**globals()))


logging.getLogger('matplotlib.font_manager').disabled = True
warnings.filterwarnings("ignore")
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 10)

In [3]:
row_accumulator = []
for filename in glob.glob('nvdcve-1.1-*.json'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for entry in nvd_data['CVE_Items']:
            cve = entry['cve']['CVE_data_meta']['ID']
            try:
                assigner = entry['cve']['CVE_data_meta']['ASSIGNER']
            except KeyError:
                assigner = 'Missing_Data'
            try:
                published_date = entry['publishedDate']
            except KeyError:
                published_date = 'Missing_Data'
            try:
                attack_vector = entry['impact']['baseMetricV3']['cvssV3']['attackVector']
            except KeyError:
                attack_vector = 'Missing_Data'
            try:
                attack_complexity = entry['impact']['baseMetricV3']['cvssV3']['attackComplexity']
            except KeyError:
                attack_complexity = 'Missing_Data'
            try:
                privileges_required = entry['impact']['baseMetricV3']['cvssV3']['privilegesRequired']
            except KeyError:
                privileges_required = 'Missing_Data'
            try:
                user_interaction = entry['impact']['baseMetricV3']['cvssV3']['userInteraction']
            except KeyError:
                user_interaction = 'Missing_Data'
            try:
                scope = entry['impact']['baseMetricV3']['cvssV3']['scope']
            except KeyError:
                scope = 'Missing_Data'
            try:
                confidentiality_impact = entry['impact']['baseMetricV3']['cvssV3']['confidentialityImpact']
            except KeyError:
                confidentiality_impact = 'Missing_Data'
            try:
                integrity_impact = entry['impact']['baseMetricV3']['cvssV3']['integrityImpact']
            except KeyError:
                integrity_impact = 'Missing_Data'
            try:
                availability_impact = entry['impact']['baseMetricV3']['cvssV3']['availabilityImpact']
            except KeyError:
                availability_impact = 'Missing_Data'
            try:
                base_score = entry['impact']['baseMetricV3']['cvssV3']['baseScore']
            except KeyError:
                base_score = '0.0'
            try:
                base_severity = entry['impact']['baseMetricV3']['cvssV3']['baseSeverity']
            except KeyError:
                base_severity = 'Missing_Data'
            try:
                exploitability_score = entry['impact']['baseMetricV3']['exploitabilityScore']
            except KeyError:
                exploitability_score = 'Missing_Data'
            try:
                impact_score = entry['impact']['baseMetricV3']['impactScore']
            except KeyError:
                impact_score = 'Missing_Data'
            try:
                cwe = entry['cve']['problemtype']['problemtype_data'][0]['description'][0]['value']
            except IndexError:
                cwe = 'Missing_Data'
            try:
                description = entry['cve']['description']['description_data'][0]['value']
            except IndexError:
                description = ''
            new_row = { 
                'CVE': cve, 
                'Published': published_date,
                'AttackVector': attack_vector,
                'AttackComplexity': attack_complexity,
                'PrivilegesRequired': privileges_required,
                'UserInteraction': user_interaction,
                'Scope': scope,
                'ConfidentialityImpact': confidentiality_impact,
                'IntegrityImpact': integrity_impact,
                'AvailabilityImpact': availability_impact,
                'BaseScore': base_score,
                'BaseSeverity': base_severity,
                'ExploitabilityScore': exploitability_score,
                'ImpactScore': impact_score,
                'CWE': cwe,
                'Description': description,
                'Assigner' : assigner
            }
            if not description.startswith('** REJECT **'): # disputed, rejected and other non issues start with '**'
                row_accumulator.append(new_row)
        nvd = pd.DataFrame(row_accumulator)
    
nvd['Published'] = pd.to_datetime(nvd['Published'])
thisyear = ((nvd['Published'] > '2022-01-01') & (nvd['Published']  < '2023-01-01'))
nvd = nvd.loc[thisyear]
nvd = nvd.sort_values(by=['Published'])
nvd = nvd.reset_index(drop=True)
nvd['BaseScore'] = pd.to_numeric(nvd['BaseScore']);
nvd['BaseScore'] = pd.to_numeric(nvd['BaseScore']);
nvd['BaseScore'] = nvd['BaseScore'].replace(0, np.NaN);
nvdcount = nvd['Published'].count()
nvdunique = nvd['Published'].nunique()
startdate = date(2022, 1, 1)
enddate  = date.today()
numberofdays = enddate - startdate 
per_day = nvdcount/numberofdays.days

In [4]:
Markdown(f"Total Number of CVEs: **{nvd['CVE'].count()}**<br />Average CVEs Per Day: **{per_day.round(2)}**<br />Average CVSS Score: **{nvd['BaseScore'].mean().round(2)}**")

Total Number of CVEs: **25093**<br />Average CVEs Per Day: **68.75**<br />Average CVSS Score: **7.19**

## Top 10 Publishing Days

In [5]:
Month_Graph = nvd['Published'].groupby(nvd.Published.dt.to_period("M")).agg('count')
Year_Graph = nvd['Published'].groupby(nvd.Published.dt.to_period("Y")).agg('count')
Week_Graph = nvd['Published'].groupby(nvd.Published.dt.to_period("W")).agg('count')
Day_Graph = nvd['Published'].groupby(nvd.Published.dt.to_period("D")).agg('count')

In [6]:
dg_df = pd.DataFrame(Day_Graph)
dg_df.columns = ['Count']
dg_df = dg_df.reset_index()
dg_df = dg_df.rename(columns={"Published" : "Date"})
dg_df['Percentage'] = ( dg_df['Count'] / 
                       dg_df['Count'].sum()) * 100
dg_df['Percentage'] = dg_df['Percentage'].round(2)
dg_df = dg_df.rename(columns={"Count" : "CVEs"})
dg_df_top_10 = dg_df.sort_values(by='CVEs', ascending=False).head(10)
dg_df_top_10 = dg_df_top_10.to_csv("dg_df_top_10.csv", index=False)
dg_df_top_10

## Publishing Month

In [7]:
mg_df = pd.DataFrame(Month_Graph)
mg_df.columns = ['Count']
mg_df = mg_df.reset_index()
mg_df = mg_df.rename(columns={"Published" : "Month"})
mg_df['Percentage'] = ( mg_df['Count'] / 
                       mg_df['Count'].sum()) * 100
mg_df['Month'] = mg_df['Month'].dt.strftime('%B')
mg_df['Percentage'] = mg_df['Percentage'].round(1)
mg_df = mg_df.rename(columns={"Count" : "CVEs"})
mg_df.to_csv("mg_df.csv", index=False)
mg_df

,Month,CVEs,Percentage
0,January,2016,8.0
1,February,1942,7.7
2,March,2059,8.2
3,April,2049,8.2
4,May,2023,8.1
5,June,2280,9.1
6,July,1969,7.8
7,August,2324,9.3
8,September,2198,8.8
9,October,1850,7.4


## Day Of The Week

In [8]:
nvd_dow = nvd
nvd_dow['Day_Of_Week'] = nvd_dow['Published'].dt.strftime('%A')
nvd_dow = nvd_dow['Day_Of_Week'].value_counts()
nvd_dow = nvd_dow.reset_index()
nvd_dow = nvd_dow.rename(columns={"index": "Day", "Day_Of_Week": "Count"}, errors="raise")
nvd_dow['Percentage'] = (nvd_dow['Count'] / 
                       nvd_dow['Count'].sum()) * 100
nvd_dow = nvd_dow.round(1)
nvd_dow = nvd_dow.rename(columns={"Count" : "CVEs"})
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
nvd_dow['Day'] = pd.Categorical(nvd_dow['Day'], categories=cats, ordered=True)
nvd_dow = nvd_dow.sort_values('Day')
nvd_dow.to_csv("nvd_dow.csv", index=False)
nvd_dow

,Day,CVEs,Percentage
3,Monday,4384,17.5
0,Tuesday,5414,21.6
4,Wednesday,4353,17.3
2,Thursday,4742,18.9
1,Friday,5231,20.8
6,Saturday,473,1.9
5,Sunday,496,2.0
